<a href="https://colab.research.google.com/github/hamletbatista/daanyc/blob/master/Automated_Intent_Classification_Using_Python_Model_Training_and_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automated Intent Classification Using Python - Model Training and Predictions


---


Check to learn more
https://www.searchenginejournal.com/automated-intent-classification-using-deep-learning-google-sheets/353910/

1. Download the prepared CSV files
2. Train our BERT-powered predictive model using Uber’s Ludwig.
3. Test predictive model on competing site


## First competitor keywords set

In [2]:
#Competitor 1 source keywords. Preprepared in the first notebook 

!wget https://raw.githubusercontent.com/hamletbatista/daanyc/master/ghostbed.csv


--2020-07-29 22:37:47--  https://raw.githubusercontent.com/hamletbatista/daanyc/master/ghostbed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348009 (340K) [text/plain]
Saving to: ‘ghostbed.csv’

ghostbed.csv        100%[===================>] 339.85K  --.-KB/s    in 0.03s   

2020-07-29 22:37:48 (10.0 MB/s) - ‘ghostbed.csv’ saved [348009/348009]



As you download files and run the training process, you can hit refresh on the left pane to see the file and download them to your computer.

![alt text](https://github.com/hamletbatista/daanyc/raw/master/refresh.png)

In [ ]:
df = pd.read_csv("ghostbed.csv")

In [ ]:
df.head(500)

,Unnamed: 0,Keyword
0,0,ghost bed
1,1,ghostbed
2,2,ghost mattress
3,3,zero gravity bed
4,4,mattress cover
...,...,...
495,495,mattress disinfectant spray
496,496,how to get blood out of a tempurpedic mattress
497,497,california king dimensions
498,498,mattress bundle


In [ ]:
#df

##Manually labeled intentions

We used heuristics in the first notebook to produce this intent dataset

In [ ]:
!wget https://raw.githubusercontent.com/hamletbatista/daanyc/master/ghost-bed-intent.csv

--2020-07-29 16:51:27--  https://raw.githubusercontent.com/hamletbatista/daanyc/master/ghost-bed-intent.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84436 (82K) [text/plain]
Saving to: ‘ghost-bed-intent.csv’

ghost-bed-intent.cs 100%[===================>]  82.46K  --.-KB/s    in 0.009s  

2020-07-29 16:51:27 (8.93 MB/s) - ‘ghost-bed-intent.csv’ saved [84436/84436]



In [ ]:
df = pd.read_csv("ghost-bed-intent.csv")

In [ ]:
df.groupby("intent").count()

,Unnamed: 0,query
intent,,
Commercial Investigation,44,44
Informational,1446,1446
Navigational,13,13
Transactional,137,137
Unknown,133,133


In [ ]:
df.head(100)

,Unnamed: 0,query,intent
0,0,ghost bed,Transactional
1,1,ghostbed,Commercial Investigation
2,2,ghost mattress,Informational
3,3,zero gravity bed,Unknown
4,4,mattress cover,Informational
...,...,...,...
95,95,purple mattress price,Informational
96,96,mattress size chart,Commercial Investigation
97,97,0 gravity bed,Informational
98,98,how much bigger is a king than a queen,Informational


## Train our BERT-powered predictive model using Uber’s Ludwig.


In [ ]:
%tensorflow_version 1.x 

import tensorflow as tf; print(tf.__version__)



1.15.2


In [ ]:
%%capture
!pip install ludwig



In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip



--2020-07-29 04:15:22--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 173.194.194.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  45.8MB/s    in 8.7s    

2020-07-29 04:15:31 (44.6 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json

In [ ]:
# create the ludwig configuration file for BERT-powered classification
#https://uber.github.io/ludwig/user_guide/#bert-encoder

template="""
input_features:
    -
        name: query
        type: text
        encoder: bert
        config_path: uncased_L-12_H-768_A-12/bert_config.json
        checkpoint_path: uncased_L-12_H-768_A-12/bert_model.ckpt
        preprocessing:
          word_tokenizer: bert
          word_vocab_file: uncased_L-12_H-768_A-12/vocab.txt
          padding_symbol: '[PAD]'
          unknown_symbol: '[UNK]'

output_features:
    -
        name: intent
        type: category
text:
        word_sequence_length_limit: 128
training:
        batch_size: 32
        learning_rate: 0.00002
        Epoch: 1

"""

with open("model_definition.yaml", "w") as f:
  f.write(template)


In [ ]:
%%capture
!pip install bert-tensorflow

In [ ]:
!ludwig experiment \
  --data_csv ghost-bed-intent.csv\
  --model_definition_file model_definition.yaml

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.2.2.8 - Experiment

Experiment name: experiment
Model name: run
Output path: results/experiment_run_0


ludwig_version: '0.2.2.8'
command: ('/usr/local/bin/ludwig experiment --data_csv intent.csv '
 '--model_definition_file model_definition.yaml')
random_seed: 42
input_data: 'intent.csv'
model_definition: {   'combiner': {'type': 'concat'},
    'input_features': [   {   'checkpoint_path': 'uncased_L-12_H-768_A-12/bert_model.ckpt',
                       

## Test predictive model on competing site

In [ ]:
#test_df = get_keywords("eachnight.com")
!wget https://raw.githubusercontent.com/hamletbatista/daanyc/master/eachnight.csv

--2020-07-29 15:32:08--  https://raw.githubusercontent.com/hamletbatista/daanyc/master/eachnight.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 362277 (354K) [text/plain]
Saving to: ‘eachnight.csv.1’

eachnight.csv.1     100%[===================>] 353.79K  --.-KB/s    in 0.06s   

2020-07-29 15:32:09 (5.69 MB/s) - ‘eachnight.csv.1’ saved [362277/362277]



Make sure the filename downloaded matches. For example "eachnight.csv.1"

In [ ]:
test_df = pd.read_csv("eachnight.csv")

In [ ]:
test_df

,Unnamed: 0,Keyword
0,0,california king bed
1,1,california king vs king
2,2,adjustable beds
3,3,cal king vs king
4,4,queen mattress
...,...,...
9996,9996,best mattress for healthy sleep
9997,9997,bed and mattress difference
9998,9998,best down mattress pad
9999,9999,mattress off gassing and pregnancy


In [ ]:
!ls results

experiment_run	experiment_run_0


Herere we also have to use the right directory folder. Typically results/experiment_run/model or results/experiment_run_0/model

In [ ]:
from ludwig.api import LudwigModel
model = LudwigModel.load("results/experiment_run_0/model")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from results/experiment_run_0/model/model_weights


If this part fails complaining about a tensorflow library, please restart the Runtime and run the lines:

```
%tensorflow_version 1.x 

import tensorflow as tf; print(tf.__version__)

test_df = pd.read_csv("eachnight.csv")

```

![alt text](https://github.com/hamletbatista/daanyc/raw/master/restart.png)

The try the above line again.

In the worse case scenario, you can uncomment the ```!wget ``` line below to download the model I trained already.

In [ ]:
#alternative
#download from https://www.dropbox.com/s/xvqpe34ocxyhk34/results-20200729T150959Z-001.zip?dl=0
#!wget https://www.dropbox.com/s/xvqpe34ocxyhk34/results-20200729T150959Z-001.zip?dl=0 && !unzip results-20200729T150959Z-001.zip?dl=0

We are only predicting on 1000 keywords out of 10000. Feel free to update the number in both cells to get more.

In [ ]:

predictions = model.predict(test_df.head(1000).rename(columns={'Keyword': 'query'} ))



In [ ]:
test_df.head(1000).join(predictions)[["Keyword", "intent_predictions"]]

,Keyword,intent_predictions
0,california king bed,Informational
1,california king vs king,Informational
2,adjustable beds,Informational
3,cal king vs king,Informational
4,queen mattress,Informational
...,...,...
995,best mattress 2020,Informational
996,how do i clean my memory foam mattress,Informational
997,best gel mattress for side sleepers,Informational
998,what sort of mattress is best for a child,Informational


This is our final output.

In [ ]:
test_df.head(1000).join(predictions)[["Keyword", "intent_predictions"]].groupby("intent_predictions").count()

,Keyword
intent_predictions,
Informational,989
Transactional,9
Unknown,2


In [ ]:
test_df.to_csv("competitor_predictions.csv")